In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings('ignore')

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
n_hidden_1 = 256
n_hidden_2 = 128
n_input = 784
n_classes = 10

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder('float', [None, n_classes])

stddev = 0.1
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=stddev)),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=stddev)),
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [22]:
def multilayer_proceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['w1']),_biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['w2']),_biases['b2']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])

In [25]:
pred = multilayer_proceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.arg_max(pred,1),tf.arg_max(y,1))
accr = tf.reduce_mean(tf.cast(corr,'float'))
init = tf.global_variables_initializer()

In [26]:
training_epochs = 20
batch_size = 100
display_step = 4
#Session
sess = tf.Session()
sess.run(init)
# mini-batch learning
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds={x:batch_xs, y:batch_ys}
        sess.run(optm,feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
    #display
    if (epoch+1) % display_step == 0:
        feeds_train = {x:batch_xs, y:batch_ys}
        feeds_test = {x:mnist.test.images, y:mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict = feeds_test)
        print("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f"%
             (epoch, training_epochs, avg_cost, train_acc, test_acc))
print('Done')

Epoch: 003/020 cost: 2.272328158 train_acc: 0.230 test_acc: 0.244
Epoch: 007/020 cost: 2.239516165 train_acc: 0.350 test_acc: 0.361
Epoch: 011/020 cost: 2.203291028 train_acc: 0.480 test_acc: 0.460
Epoch: 015/020 cost: 2.161594904 train_acc: 0.480 test_acc: 0.529
Epoch: 019/020 cost: 2.112211565 train_acc: 0.520 test_acc: 0.572
Done
